In [10]:
# Imports 
import pandas as pd
import itertools 
import networkx as nx
# import matplotlib.pyplot as plt
# import pytrends
# import musicbrainzngs as m
import numpy as np
import requests
import sys

# from pytrends.request import TrendReq
from os import path
from PIL import Image
# from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [11]:
def get_access_token(client_id, client_secret):
    """
    Get the access token from Spotify
    """
    body_params = {'grant_type': "client_credentials"}
    url = 'https://accounts.spotify.com/api/token'

    response = requests.post(url, data=body_params, auth=(client_id, client_secret))
    if response.status_code == 200:
        return response.json()['access_token']
    else:
        sys.exit("Failed to get access token. Is your client_id and client_secret correct?")

In [19]:
access_ = get_access_token("338302b9161d4941a728cb84d47889ed","c9954db6ff16412ea9036d74ebaa8a47")

In [20]:
access = "Bearer " + access_

In [15]:
def find_artist_genres(artist_name):
    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'Authorization': access}
    params = {
        'q': artist_name,
        'type': 'artist',
        'limit': '1',
    }

    response = requests.get('https://api.spotify.com/v1/search', params=params, headers=headers)
    
    return response.json()['artists']['items'][0]['genres']

In [16]:
find_artist_genres("Lady Gaga")

['art pop', 'dance pop', 'pop']

In [41]:
import ciso8601
import time
import re
def separate_artists(df):
    split_string = ' and |, | AND | & | featuring | FEATURING | x | X | feat | ft |,| with | feauring | feat. | / '
    ret_list = []
    res_check = set()
    for index, colab in df.iterrows():
        date = ciso8601.parse_datetime(colab['date'])
        res = [time.mktime(date.timetuple())]
        artists = re.split(split_string, colab['artist'], flags=re.IGNORECASE)
        if tuple(artists) not in res_check:
            res_check.add(tuple(artists))
            res.append(artists)
            ret_list.append(res)
    return ret_list

In [69]:
df['song']

0           All I Want For Christmas Is You
1         Rockin' Around The Christmas Tree
2                          Jingle Bell Rock
3                   A Holly Jolly Christmas
4                                Easy On Me
                        ...                
114895                            L.A. Song
114896                            Gotta Man
114897                    The Quittin' Kind
114898                                Steam
114899                          NAStradamus
Name: song, Length: 114900, dtype: object

In [49]:
df = pd.read_csv("../data/billboard.csv", index_col = 0)
df = df[df.date >= "2000-01-01"]

separated_artists = separate_artists(df)
collabs = [y for [x,y] in separated_artists if len(y)>1]

In [ ]:
df = pd.read_csv("../docs/src/chartview/data/data.csv")

In [69]:
def find_uri(artist, track):
    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'Authorization': access,}

    params = {
         # 'q': '"' + track + '" "' + artist + '"',
         'q': "track: " + track + " artist:" + artist,
        'type': 'track',
        'limit': '1',
        'market': 'US'
    }

    response = requests.get('https://api.spotify.com/v1/search', params=params, headers=headers)
    return response.json()['tracks']['items'][0]['uri'].split(":")[-1]

In [57]:
df

,_uri,id,_date,_song,_artist,_peak-rank,_weeks-on-board,_genres,_track,_uri.1,year,keywords
0,1m2xMsxbtxv21Brome189p,I_Wanna_Know_by_Joe,2000-01-01,I Wanna Know,Joe,4,44,"['american', 'bebop', 'death by liver cancer',...",I Wanna Know,1m2xMsxbtxv21Brome189p,2000,"american, bebop, death by liver cancer, jazz"
1,3BsaRV5QIulYz2lV9WWa8T,Show_Me_The_Meaning_Of_Being_Lonely_by_Backstr...,2000-01-01,Show Me The Meaning Of Being Lonely,Backstreet Boys,6,24,"['american', 'boy band', 'boy group', 'dance-p...",Show Me The Meaning Of Being Lonely,3BsaRV5QIulYz2lV9WWa8T,2000,"american, boy band, boy group, dance-pop, elec..."
2,0bYg9bo50gSsH3LtXe2SQn,All_I_Want_For_Christmas_Is_You_by_Mariah_Carey,2000-01-08,All I Want For Christmas Is You,Mariah Carey,1,50,"['adult contemporary', 'american', 'christmas ...",All I Want For Christmas Is You,0bYg9bo50gSsH3LtXe2SQn,2000,"adult contemporary, american, christmas music,..."
3,1sxUaLi0G2vB7dl4ogtCxH,Never_Let_You_Go_by_Third_Eye_Blind,2000-01-22,Never Let You Go,Third Eye Blind,14,22,"['alternative rock', 'classic pop and rock', '...",Never Let You Go,1sxUaLi0G2vB7dl4ogtCxH,2000,"alternative rock, classic pop and rock, pop ro..."
4,4Y8q64VnhD0vFYy9g2WFpi,It_Feels_So_Good_by_Sonique,2000-01-22,It Feels So Good,Sonique,8,24,"['breakbeat', 'british', 'dance-pop', 'english...",It Feels So Good,4Y8q64VnhD0vFYy9g2WFpi,2000,"breakbeat, british, dance-pop, english, euroda..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1010,Unknown,Escape_Plan_by_Travis_Scott,2021-11-20,Escape Plan,Travis Scott,11,3,"['english', 'hip hop', 'rap', 'relic inn', 'tr...",Unknown,Unknown,2021,"english, hip hop, rap, relic inn, trap, travis..."
1011,Unknown,Oh_My_God_by_Adele,2021-12-04,Oh My God,Adele,5,5,"['adele', 'adult alternative pop/rock', 'adult...",Unknown,Unknown,2021,"adele, adult alternative pop/rock, adult conte..."
1012,Unknown,abcdefu_by_GAYLE,2021-12-04,abcdefu,GAYLE,15,5,Unknown,Unknown,Unknown,2021,Unknown
1013,Unknown,I_Hate_U_by_SZA,2021-12-18,I Hate U,SZA,7,3,"['american', 'chillwave', 'electronic']",Unknown,Unknown,2021,"american, chillwave, electronic"


In [70]:
for i, row in df.iterrows():
    if row['_uri'] == "Unknown":
        try: 
            uri = find_uri(row["_song"], row["_artist"] )
            df.at[i, '_uri'] = uri
        except IndexError:
            print(row["_song"], row["_artist"])

Addictive Truth Hurts Featuring Rakim
Picture Kid Rock Featuring Sheryl Crow Or Allison Moorer
You Lloyd Featuring Lil Wayne
Dream Big David Cook
Sugar Flo Rida Featuring Wynter
Drunk In Love Beyonce Featuring Jay Z
Walk On Water Eminem Featuring Beyonce
Let It Fly Lil Wayne Featuring Travis Scott
I Hope Gabby Barrett Featuring Charlie Puth
Life Is Good Future Featuring Drake
Godzilla Eminem Featuring Juice WRLD
Intentions Justin Bieber Featuring Quavo
Rockstar DaBaby Featuring Roddy Ricch
Pain 1993 Drake Featuring Playboi Carti
Chicago Freestyle Drake Featuring Giveon
Rags2Riches Rod Wave Featuring ATR Son Son
Popstar DJ Khaled Featuring Drake
Greece DJ Khaled Featuring Drake
Exile Taylor Swift Featuring Bon Iver
WAP Cardi B Featuring Megan Thee Stallion
Mood 24kGoldn Featuring iann dior
Laugh Now Cry Later Drake Featuring Lil Durk
Ice Cream BLACKPINK X Selena Gomez
Holy Justin Bieber Featuring Chance The Rapper
Levitating Dua Lipa Featuring DaBaby
Back In Blood Pooh Shiesty Featuring

In [55]:
"spotify:track:5dXWFMwD7I7zXsInONVl0H".split(":")[-1]

['spotify', 'track', '5dXWFMwD7I7zXsInONVl0H']

In [50]:
for i in collabs:
    try :
        i.remove('R. Kelly')
    except ValueError:
        pass

In [51]:
unique_artists = list(np.unique([artist for l in collabs for artist in l]))

In [52]:
unique_artists;
artists_by_id = dict({a:b for a,b in zip(unique_artists,list(range(0,len(unique_artists))))})
artists_by_id = {key:[value,0.0] for (key,value) in artists_by_id.items()}
artists_by_id.pop("R. Kelly")

KeyError: 'R. Kelly'

In [53]:
for l in collabs:
    for artist in l:
        artists_by_id[artist][1] +=1

In [54]:
top100=sorted(artists_by_id.items(), key=lambda x: x[1][1], reverse=True)[:100];
top100=dict(top100);
top100

{'Lil Wayne': [957, 109.0],
 'Drake': [459, 108.0],
 'Nicki Minaj': [1186, 81.0],
 'Chris Brown': [315, 71.0],
 'Kanye West': [831, 67.0],
 'Future': [570, 57.0],
 'Young Thug': [1760, 57.0],
 'Lil Baby': [936, 53.0],
 'Rick Ross': [1332, 48.0],
 'Justin Bieber': [811, 47.0],
 'Travis Scott': [1645, 47.0],
 'Eminem': [495, 44.0],
 'Big Sean': [176, 41.0],
 'DJ Khaled': [379, 40.0],
 'Ludacris': [993, 40.0],
 'Meek Mill': [1074, 40.0],
 'T-Pain': [1502, 40.0],
 '2 Chainz': [8, 36.0],
 'Lil Uzi Vert': [956, 33.0],
 'T.I.': [1505, 33.0],
 '21 Savage': [10, 32.0],
 'DaBaby': [390, 31.0],
 'Juice WRLD': [803, 31.0],
 'Pitbull': [1258, 31.0],
 'Akon': [51, 30.0],
 'Lil Durk': [941, 30.0],
 'Rihanna': [1335, 30.0],
 'The Weeknd': [1595, 30.0],
 'Ty Dolla $ign': [1664, 30.0],
 'Cardi B': [265, 29.0],
 'Tyga': [1665, 29.0],
 'Ariana Grande': [99, 28.0],
 'Wiz Khalifa': [1712, 28.0],
 'Gunna': [618, 27.0],
 'Kendrick Lamar': [858, 27.0],
 'Snoop Dogg': [1445, 27.0],
 'Usher': [1678, 27.0],
 'Pol

In [35]:
find_artist_genres('Shakira')

['colombian pop', 'dance pop', 'latin', 'latin pop', 'pop']

In [55]:
def determine_genre(subgenres) :
    count_genres = dict(zip(aggregate_genres.keys(), np.zeros(len(aggregate_genres.keys()))))
    find_one = False
    for subgenre in subgenres :
        for genre, subgrs in aggregate_genres.items():
            if subgenre in subgrs:
                find_one = True
                count_genres[genre] +=1
    if (find_one):
        return sorted(count_genres.items(), key=lambda x: x[1])[-1][0]
    else :
        return np.NaN

In [56]:
# Source : https://github.com/kevinschaich/billboard but completed with genres suggested by Spotify API
aggregate_genres = {
    "rock": ["symphonic rock", "jazz-rock", "heartland rock", "rap rock", "garage rock", 
           "folk-rock", "roots rock", "adult alternative pop rock", "rock roll", "punk rock", 
           "arena rock", "pop-rock", "glam rock", "southern rock", "indie rock", "funk rock", 
           "country rock", "piano rock", "art rock", "rockabilly", "acoustic rock", "progressive rock", 
           "folk rock", "psychedelic rock", "rock & roll", "blues rock", "alternative rock", 
           "rock and roll", "soft rock", "rock and indie", "hard rock", "pop/rock", "pop rock", 
           "rock", "classic pop and rock", "psychedelic", "british psychedelia", "punk", "metal", "heavy metal", 
             "brutal death metal", 'deep talent show'],
"alternative/indie": ["adult alternative pop rock", "alternative rock", "alternative metal", 
                       "alternative", "lo-fi indie", "indie", "indie folk", "indietronica", 
                       "indie pop", "indie rock", "rock and indie"],
"electronic/dance": ['big room', 'edm', "dance and electronica", "pop dance", "electro house", "electronic", "electropop", 
                      "progressive house", "hip house", "house", "eurodance", "dancehall", "dance", "funk carioca"],
"soul": ["psychedelic soul", "deep soul", "neo-soul", "neo soul", "southern soul", 
          "smooth soul", "blue-eyed soul", "soul and reggae", "soul", "funk"],
"classical/soundtrack": ["classical", "orchestral", "film soundtrack", "composer"],
"pop": ["country-pop","urban contemporary", "latin pop", "classical pop", "pop-metal", "orchestral pop", 
         "instrumental pop", "indie pop", "sophisti-pop", "pop punk", "pop reggae", "noah cyrus"
         "britpop", "traditional pop", "power pop", "sunshine pop", "baroque pop", "alternative r&b",
         "synthpop", "art pop", "teen pop", "psychedelic pop", "folk pop", "country pop", "pop soul",
         "pop and chart", "dance-pop", "pop", "top 40", "american idol", 'breakeven', 'm2m', 'dance pop'],
"hip-hop/rap": ["conscious hip hop", "east coast hip hop", "hardcore hip hop", 'oakland hip hop', 'pop rap'
                 "west coast hip hop", "hiphop", "southern hip hop", "hip-hop", 
                 "hip hop", "hip hop rnb and dance hall", "contemporary r b", 
                 "gangsta rap", 'gangster rap', "rapper", "rap", "mumble rapper", 'chicago rap', 'melodic rap', 'brooklyn drill', 'drill',
               'baton rouge rap', 'trap', 'southern hip hop', 'miami hip hop', 'dirty south rap'],
"rnb": ["rnb", "rhythm & blues","r&b", "blues", "contemporary rnb", 
                 "contemporary r&b", "rhythm and blues"],
"disco": ["disco"],
"swing":  ["swing"],
"folk": ["contemporary folk", "folk", "bachata", "romanian"],
"country": ["country rock", "country-pop", "country pop", "contemporary country", "country"],
"jazz": ["vocal jazz", "jazz", "jazz-rock"],
"religious": ["christian", "christmas music", "gospel"],
"blues": ["delta blues", "rock blues", "urban blues", "electric blues",
           "acoustic blues", "soul blues", "country blues", "jump blues", 
           "classic rock", "blues rock", "jazz and blues", "piano blues", 
           "british blues", "british rhythm & blues", "rhythm and blues", "blues", "blues rock", "rhythm & blues"],
"reggae": ["reggae", "reggae fusion", "roots reggae", "reggaeton", "pop reggae", "soul and reggae"]}

In [57]:
for key, elem in top100.items():
    elem.append(determine_genre(find_artist_genres(key)))

In [58]:
find_artist_genres("David Guetta")

['big room', 'dance pop', 'edm', 'pop', 'pop dance', 'pop rap']

In [59]:
top100

{'Lil Wayne': [957, 109.0, 'hip-hop/rap'],
 'Drake': [459, 108.0, 'hip-hop/rap'],
 'Nicki Minaj': [1186, 81.0, 'pop'],
 'Chris Brown': [315, 71.0, 'pop'],
 'Kanye West': [831, 67.0, 'hip-hop/rap'],
 'Future': [570, 57.0, 'hip-hop/rap'],
 'Young Thug': [1760, 57.0, 'hip-hop/rap'],
 'Lil Baby': [936, 53.0, 'hip-hop/rap'],
 'Rick Ross': [1332, 48.0, 'hip-hop/rap'],
 'Justin Bieber': [811, 47.0, 'pop'],
 'Travis Scott': [1645, 47.0, 'hip-hop/rap'],
 'Eminem': [495, 44.0, 'hip-hop/rap'],
 'Big Sean': [176, 41.0, 'hip-hop/rap'],
 'DJ Khaled': [379, 40.0, 'hip-hop/rap'],
 'Ludacris': [993, 40.0, 'hip-hop/rap'],
 'Meek Mill': [1074, 40.0, 'hip-hop/rap'],
 'T-Pain': [1502, 40.0, 'hip-hop/rap'],
 '2 Chainz': [8, 36.0, 'hip-hop/rap'],
 'Lil Uzi Vert': [956, 33.0, 'hip-hop/rap'],
 'T.I.': [1505, 33.0, 'hip-hop/rap'],
 '21 Savage': [10, 32.0, 'hip-hop/rap'],
 'DaBaby': [390, 31.0, 'hip-hop/rap'],
 'Juice WRLD': [803, 31.0, 'hip-hop/rap'],
 'Pitbull': [1258, 31.0, 'pop'],
 'Akon': [51, 30.0, 'pop'],

In [61]:
selected_genres = []

for k,v in top100.items():
    if ((v[2] not in selected_genres)):
        selected_genres.append(v[2])

In [63]:
selected_genres

['hip-hop/rap', 'pop', 'rnb', 'reggae', 'electronic/dance']

In [60]:
top100['David Guetta'] = [410, 21.0, 'electronic/dance']

In [62]:
top100

{'Lil Wayne': [957, 109.0, 'hip-hop/rap'],
 'Drake': [459, 108.0, 'hip-hop/rap'],
 'Nicki Minaj': [1186, 81.0, 'pop'],
 'Chris Brown': [315, 71.0, 'pop'],
 'Kanye West': [831, 67.0, 'hip-hop/rap'],
 'Future': [570, 57.0, 'hip-hop/rap'],
 'Young Thug': [1760, 57.0, 'hip-hop/rap'],
 'Lil Baby': [936, 53.0, 'hip-hop/rap'],
 'Rick Ross': [1332, 48.0, 'hip-hop/rap'],
 'Justin Bieber': [811, 47.0, 'pop'],
 'Travis Scott': [1645, 47.0, 'hip-hop/rap'],
 'Eminem': [495, 44.0, 'hip-hop/rap'],
 'Big Sean': [176, 41.0, 'hip-hop/rap'],
 'DJ Khaled': [379, 40.0, 'hip-hop/rap'],
 'Ludacris': [993, 40.0, 'hip-hop/rap'],
 'Meek Mill': [1074, 40.0, 'hip-hop/rap'],
 'T-Pain': [1502, 40.0, 'hip-hop/rap'],
 '2 Chainz': [8, 36.0, 'hip-hop/rap'],
 'Lil Uzi Vert': [956, 33.0, 'hip-hop/rap'],
 'T.I.': [1505, 33.0, 'hip-hop/rap'],
 '21 Savage': [10, 32.0, 'hip-hop/rap'],
 'DaBaby': [390, 31.0, 'hip-hop/rap'],
 'Juice WRLD': [803, 31.0, 'hip-hop/rap'],
 'Pitbull': [1258, 31.0, 'pop'],
 'Akon': [51, 30.0, 'pop'],

In [72]:
find_artist_genres("Usher")

['atl hip hop',
 'contemporary r&b',
 'dance pop',
 'pop',
 'r&b',
 'south carolina hip hop',
 'urban contemporary']

In [63]:
selected_genres = []

for k,v in top100.items():
    if ((v[2] not in selected_genres)):
        selected_genres.append(v[2])
        
selected_genres

['hip-hop/rap', 'pop', 'reggae', 'electronic/dance', 'rnb', 'rock']

In [64]:
import os 

f = open("nodes.json", "w")

f.write("[")

color = ""
for artist, elems in top100.items():
    genre = elems[2]

    if genre == 'hip-hop/rap':
         color = "#001219"
    elif genre == 'pop':
        color = "#005f73"
    elif genre == 'rnb':
        color = "#e9d8a6"        
    elif genre == 'reggae':
        color = "#ee9b00"
    elif genre == 'electronic/dance':
        color = "#bb3e03"
    elif genre == 'rock':
        color = "#9b2226"
    
    f.write('{"id": ' + str(elems[0]) )
    f.write(', "label": "' + artist  )
    f.write('", "group": "' + elems[2] + '", "color": "' + color +'"},\n')

f.seek(f.tell() - 1, os.SEEK_SET)
f.write('')
f.seek(f.tell() - 1, os.SEEK_SET)
f.write('')
f.seek(f.tell() - 1, os.SEEK_SET)
f.write('')

f.write("]")
f.close()

In [65]:
f = open("edges.json", "w")

f.write("[")

for pair in list(itertools.combinations(list(top100.keys()), 2)):
    for collab in collabs:
        if pair[0] in collab and pair[1] in collab:
                f.write('{"from": ' + str(artists_by_id[pair[0]][0]) +', "to": ' + str(artists_by_id[pair[1]][0]) + '},\n')
                break
        
f.seek(f.tell() - 1, os.SEEK_SET)
f.write('')
f.seek(f.tell() - 1, os.SEEK_SET)
f.write('')
f.seek(f.tell() - 1, os.SEEK_SET)
f.write('')

f.write("]")
f.close()

In [ ]:
most_collab_artists = [a for (a,b) in artists_by_id.items()]

In [ ]:
most_collab_artists

In [ ]:
list(itertools.combinations(collabs[6], 2))[0][0]

In [ ]:
artists_by_id['J. Cole']

In [ ]:
included_artists = most_collab_artists.copy()
included_artists = set(included_artists)

for art in most_collab_artists:
    for collab in collabs: 
        if art in collab:
            included_artists = included_artists.union(set(collab))
included_artists = list(included_artists)

In [ ]:
artists_by_id = dict({a:b for a,b in zip(included_artists,list(range(0,len(included_artists))))})
artists_by_id 

In [ ]:
f = open("nodes.txt", "w")

f.write("var nodes = new vis.DataSet([\n")

for artist, i in top100.items():
    f.write("\t{id: "+str(i[0])+", label: '"+artist+"'},\n") 
        
f.write("]);")
f.close()

In [ ]:
'Lil Wayne' in ['Lil Wayne', 'asdsd'] and 'Li' in ['Lil Wayne', 'asdsd']

In [ ]:
 for pair in list(itertools.combinations(list(top100.keys()), 2)):
        print(pair)

In [ ]:
artists_by_id['Lil Wayne'][0]

In [ ]:
f = open("edges.txt", "w")


f.write("var edges = new vis.DataSet([\n")

for pair in list(itertools.combinations(list(top100.keys()), 2)):
    for collab in collabs:
        if pair[0] in collab and pair[1] in collab:
                f.write("\t{from: "+str(artists_by_id[pair[0]][0])+', to: '+str(artists_by_id[pair[1]][0])+"},\n")
                break
        
f.write("]);")
f.close()

In [ ]:
collabs[6]

In [ ]:
f = open("hey.txt", "w")

f.write("series: [")

for genre in aggregate_genres.keys():
    f.write("\t{\n")
    f.write("\tname : "+"'"+genre+"'"+",\n")
    f.write("\tdata: [")
    
    for subgenre in aggregate_genres[genre] : 
        f.write("{\n")
        f.write("\t\tname: "+"'"+subgenre+"',\n")
        f.write("\t\tvalue: "+str(subgenres_popularity[subgenre])+"\n")
        f.write("\t},")
    
    f.write('\t]\n')   #  closing data 
    f.write("\t},\n")

f.write("]")

f.close()

In [ ]:
# Source : https://github.com/kevinschaich/billboard 
aggregate_genres = {
    "rock": ["symphonic rock", "jazz-rock", "heartland rock", "rap rock", "garage rock", 
           "folk-rock", "roots rock", "adult alternative pop rock", "rock roll", "punk rock", 
           "arena rock", "pop-rock", "glam rock", "southern rock", "indie rock", "funk rock", 
           "country rock", "piano rock", "art rock", "rockabilly", "acoustic rock", "progressive rock", 
           "folk rock", "psychedelic rock", "rock & roll", "blues rock", "alternative rock", 
           "rock and roll", "soft rock", "rock and indie", "hard rock", "pop/rock", "pop rock", 
           "rock", "classic pop and rock", "psychedelic", "british psychedelia", "punk", "metal", "heavy metal", 
             "brutal death metal"],
"alternative/indie": ["adult alternative pop rock", "alternative rock", "alternative metal", 
                       "alternative", "lo-fi indie", "indie", "indie folk", "indietronica", 
                       "indie pop", "indie rock", "rock and indie"],
"electronic/dance": ["dance and electronica", "electro house", "electronic", "electropop", 
                      "progressive house", "hip house", "house", "eurodance", "dancehall", "dance", "trap", "funk carioca"],
"soul": ["psychedelic soul", "deep soul", "neo-soul", "neo soul", "southern soul", 
          "smooth soul", "blue-eyed soul", "soul and reggae", "soul", "funk"],
"classical/soundtrack": ["classical", "orchestral", "film soundtrack", "composer"],
"pop": ["country-pop", "latin pop", "classical pop", "pop-metal", "orchestral pop", 
         "instrumental pop", "indie pop", "sophisti-pop", "pop punk", "pop reggae", "noah cyrus"
         "britpop", "traditional pop", "power pop", "sunshine pop", "baroque pop", "alternative r&b",
         "synthpop", "art pop", "teen pop", "psychedelic pop", "folk pop", "country pop", 
         "pop rap", "pop soul", "pop and chart", "dance-pop", "pop", "top 40", "american idol", 'breakeven', 'm2m'],
"hip-hop/rnb": ["conscious hip hop", "east coast hip hop", "hardcore hip hop", 
                 "west coast hip hop", "hiphop", "southern hip hop", "hip-hop", 
                 "hip hop", "hip hop rnb and dance hall", "contemporary r b", 
                 "gangsta rap", "rapper", "rap", "rhythm and blues", "contemporary rnb", 
                 "contemporary r&b", "rnb", "rhythm & blues","r&b", "blues", "mumble rapper", 'michael jackson'],
"disco": ["disco"],
"swing":  ["swing"],
"folk": ["contemporary folk", "folk", "bachata", "latin", "romanian"],
"country": ["country rock", "country-pop", "country pop", "contemporary country", "country"],
"jazz": ["vocal jazz", "jazz", "jazz-rock"],
"religious": ["christian", "christmas music", "gospel"],
"blues": ["delta blues", "rock blues", "urban blues", "electric blues",
           "acoustic blues", "soul blues", "country blues", "jump blues", 
           "classic rock", "blues rock", "jazz and blues", "piano blues", 
           "british blues", "british rhythm & blues", "rhythm and blues", "blues", "blues rock", "rhythm & blues"],
"reggae": ["reggae fusion", "roots reggae", "reggaeton", "pop reggae", "reggae", "soul and reggae"]}

In [ ]:
all_subgenres=set([i for i in list(artist_genres_df.genres.values) for i in i])

In [ ]:
[i for i in artist_genres_df.loc[artist_genres_df['artist'] == 'Lil Wayne'].genres for i in i]

In [ ]:
df = pd.read_csv("../data/billboard.csv", index_col = 0)
df = df[df.date >= "2000-01-01"]
df

In [ ]:
def determine_genre(subgenres) :
    for subgenre in subgenres :
        for genre, subgrs in aggregate_genres.items():
            if subgenre in subgrs:
                return genre
    return np.NaN

In [37]:
def determine_genre(subgenres) :
    for subgenre in subgenres :
        for genre, subgrs in aggregate_genres.items():
            if subgenre in subgrs:
                return genre
    return np.NaN

```
def from_artist_to_subgenres(artist):
    subgenres = artist_genres_df.loc[artist_genres_df['artist'] == artist].genres 
    if subgenres.empty:
        return np.NaN
    else :
        return [i for i in subgenres for i in i]
```

In [ ]:
df['subgenres'] = df['artist'].map(from_artist_to_subgenres)

In [ ]:
df

<h4>Weight formula applied to determine the subgenre popularity </h4>



Popularity of each subgenre is initialized at 0.  
$s$ is a linear score given accordingly to the rank of the subgenre occurrence.


Then, for each occurrence of the genre in the charts we have :  
$ \widehat{popularity_{subgenre}} = popularity_{subgenre} +s$
  
 where $s = 1 / rank $

In [ ]:
subgenres_popularity = dict({e:0 for e in [i for i in list(aggregate_genres.values()) for i in i]})

In [ ]:
for index, row in df.loc[pd.notna(df['subgenres'])].iterrows():
    for subg in row['subgenres']:
        try : 
            subgenres_popularity[subg] += 1/row['rank']
        except KeyError:
            continue
        
    
    # print(index, row['rank'], row['subgenres'])

In [ ]:
# subgenres_popularity = {key:value for (key,value) in subgenres_popularity.items() if value>0}

In [ ]:
f = open("hey.txt", "w")

f.write("series: [")

for genre in aggregate_genres.keys():
    f.write("\t{\n")
    f.write("\tname : "+"'"+genre+"'"+",\n")
    f.write("\tdata: [")
    
    for subgenre in aggregate_genres[genre] : 
        f.write("{\n")
        f.write("\t\tname: "+"'"+subgenre+"',\n")
        f.write("\t\tvalue: "+str(subgenres_popularity[subgenre])+"\n")
        f.write("\t},")
    
    f.write('\t]\n')   #  closing data 
    f.write("\t},\n")

f.write("]")

f.close()

In [ ]:
separate_artists(artist_genres_df)